In [ ]:
import requests
import pandas as pd
import time

# Set the API endpoint URL
url = "https://api-optimistic.etherscan.io/api"

# Read the list of addresses from the file
with open("addresses.txt", "r") as f:
    addresses = f.read().splitlines()

# Split the list of addresses into batches of size 20
address_batches = [addresses[i:i+20] for i in range(0, len(addresses), 20)]

# Set the query parameters
# https://docs.optimism.etherscan.io/api-endpoints/accounts#get-ether-balance-for-multiple-addresses-in-a-single-call
payload = {
    "module": "account",
    "action": "balancemulti",
    "tag": "latest",
    # Add your key here
    "apikey": ""
}

# Initialize an empty pandas DataFrame to store the data
df = pd.DataFrame()

# Send an HTTP GET request for each batch of addresses
for i, batch in enumerate(address_batches):
    # Add the batch of addresses to the query parameters
    payload["address"] = ",".join(batch)
    
    # Send the HTTP GET request
    response = requests.get(url, params=payload)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Check if there are any errors
        if data["status"] == "1":
            # Print the balances of each account in the batch
            for account in data["result"]:
                df = df.append(pd.json_normalize(data["result"]))
                time.sleep(0.25)
        else:
            print(f"Error: {data['message']}")
    else:
        print(f"HTTP error {response.status_code}")
    

In [41]:
# Convert the "balance" column from wei to ether
df["balance"] = df["balance"].astype(float) / 10**18

# Create a new column indicating whether the balance is greater than zero
df["has_balance"] = df["balance"] > 0

# Calculate the percentage of addresses with a balance greater than zero
percentage_positive_balance = df["has_balance"].mean() * 100

# Print the distribution of addresses with a balance greater than zero
print(f"Out of {len(df)} addresses, {df['has_balance'].sum()} ({percentage_positive_balance:.2f}%) have a balance greater than 0.")






Out of 160 addresses, 112 (70.00%) have a balance greater than 0.
